In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

### Walmart Sales Data

For the independent practice, we will analyze the weekly sales data from Walmart over a two year period from 2010 to 2012.

The data is again separated by store and by department, but we will focus on analyzing one store for simplicity.

The data includes:

- Store - the store number
- Dept - the department number
- Date - the week
- Weekly_Sales -  sales for the given department in the given store
- IsHoliday - whether the week is a special holiday week


#### Loading the data and setting the DateTimeIndex

In [3]:
data = pd.read_csv('../../DS-SF-32/lessons/lesson-17/train.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data.head()

,Store,Dept,Weekly_Sales,IsHoliday
Date,,,,
2010-02-05,1,1,24924.50,False
2010-02-12,1,1,46039.49,True
2010-02-19,1,1,41595.55,False
2010-02-26,1,1,19403.54,False
2010-03-05,1,1,21827.90,False


#### Filter the dataframe to Store 1 sales

In [17]:
# Filter the dataframe to Store 1 sales
s1df = data[data['Store']==1]
s1df.head(2)

,Store,Dept,Weekly_Sales,IsHoliday
Date,,,,
2010-02-05,1,1,24924.50,False
2010-02-12,1,1,46039.49,True


#### Aggregate over departments to compute the total sales per store, and aggregate over departments within store 1

In [45]:
# aggregate over departments to compute the total sales per store.
data.groupby('Store')['Weekly_Sales'].sum().head(3)

Store
1    2.224028e+08
2    2.753824e+08
3    5.758674e+07
Name: Weekly_Sales, dtype: float64

In [46]:
s1df.groupby('Dept')['Weekly_Sales'].sum().head(5)

Dept
1    3219405.18
2    6592598.93
3    1880518.36
4    5285874.09
5    3468885.58
Name: Weekly_Sales, dtype: float64

#### Plot the rolling_mean for `Weekly_Sales`. What general trends do you observe?

In [47]:
# aggregate over departments to compute the total sales per week for store 1.
weekly_sums = s1df[['Weekly_Sales']].resample('W').sum()
weekly_sums.head(3)

,Weekly_Sales
Date,
2010-02-07,1643690.90
2010-02-14,1641957.44
2010-02-21,1611968.17


In [48]:
rolling_weekly_sales = weekly_sums.rolling(window=4, center=False)
rolling_weekly_sales.mean().head(10)

,Weekly_Sales
Date,
2010-02-07,NaN
2010-02-14,NaN
2010-02-21,NaN
2010-02-28,1.576836e+06
2010-03-07,1.554615e+06
2010-03-14,1.504011e+06
2010-03-21,1.469148e+06
2010-03-28,1.467823e+06
2010-04-04,1.477864e+06


#### Compute the 1, 2, 52 autocorrelations for `Weekly_Sales` and/or create an autocorrelation plot.

In [69]:
# find correlation with previous week
weekly_sums.head(3)
# print weekly_sums.autocorr(lag=1)

,Weekly_Sales
Date,
2010-02-07,1643690.90
2010-02-14,1641957.44
2010-02-21,1611968.17


In [70]:
# get time series
weekly_sales = weekly_sums['Weekly_Sales']
weekly_sales.head(3)

Date
2010-02-07    1643690.90
2010-02-14    1641957.44
2010-02-21    1611968.17
Freq: W-SUN, Name: Weekly_Sales, dtype: float64

In [44]:
# Compute the 1, 2, 52 autocorrelations for Weekly_Sales
print weekly_sales.autocorr(lag=1)
print weekly_sales.autocorr(lag=2)
print weekly_sales.autocorr(lag=52)


0.302158279411
0.127413130554
0.895376029478


#### Split the weekly sales data in a training and test set - using 75% of the data for training

In [84]:
print s1df.shape
num_rows = s1df.shape[0]
training_cutoff = int(s1df.shape[0]*0.75)
x = s1df.head(training_cutoff)['Weekly_Sales'].copy()
x = x.to_frame()
print x.shape
x.head(3)

(10244, 4)
(7683, 1)


,Weekly_Sales
Date,
2010-02-05,24924.50
2010-02-12,46039.49
2010-02-19,41595.55


In [85]:
y = s1df.tail(num_rows-training_cutoff)['Weekly_Sales']
print y.shape
2561 + 7683

(2561,)


10244

#### Create an AR(1) model on the training data and compute the mean absolute error of the predictions.

In [86]:
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

In [89]:
# Create an AR(1) model on the training data
from statsmodels.tsa.arima_model import ARIMA

ar_coeff = 1 # p 
ma_coeff = 0 # q
d = 0

model = ARIMA(x, (ar_coeff, d, ma_coeff)).fit()
print model.summary()



                              ARMA Model Results                              
Dep. Variable:           Weekly_Sales   No. Observations:                 7683
Model:                     ARMA(1, 0)   Log Likelihood              -76604.285
Method:                       css-mle   S.D. of innovations           5174.660
Date:                Tue, 11 Apr 2017   AIC                         153214.570
Time:                        21:16:47   BIC                         153235.410
Sample:                    02-05-2010   HQIC                        153221.718
                         - 10-08-2010                                         
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
const               1.372e+04   1227.307     11.178      0.000      1.13e+04  1.61e+04
ar.L1.Weekly_Sales     0.9520      0.003    273.118      0.000         0.945     0.959
                    

In [94]:
# compute the mean absolute error of the predictions.
# look up ARMA model to predict
# use mean_absolute_error for analysis

#### Plot the residuals - where are their significant errors.

In [ ]:
# TODO

In [ ]:
# TODO

#### Compute and AR(2) model and an ARMA(2, 2) model - does this improve your mean absolute error on the held out set.

In [90]:
# AR(2)
ar_coeff = 2 # p 
ma_coeff = 0 # q
d = 0

model = ARIMA(x, (ar_coeff, d, ma_coeff)).fit()
print model.summary()

                              ARMA Model Results                              
Dep. Variable:           Weekly_Sales   No. Observations:                 7683
Model:                     ARMA(2, 0)   Log Likelihood              -76421.099
Method:                       css-mle   S.D. of innovations           5052.708
Date:                Tue, 11 Apr 2017   AIC                         152850.198
Time:                        21:19:09   BIC                         152877.985
Sample:                    02-05-2010   HQIC                        152859.728
                         - 10-08-2010                                         
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
const               1.372e+04   1527.392      8.981      0.000      1.07e+04  1.67e+04
ar.L1.Weekly_Sales     0.7463      0.011     66.934      0.000         0.724     0.768
ar.L2.Weekly_Sales  

In [91]:
# ARMA(2, 2)
ar_coeff = 2 # p 
ma_coeff = 2 # q
d = 0

model = ARIMA(x, (ar_coeff, d, ma_coeff)).fit()
print model.summary()

                              ARMA Model Results                              
Dep. Variable:           Weekly_Sales   No. Observations:                 7683
Model:                     ARMA(2, 2)   Log Likelihood              -76240.379
Method:                       css-mle   S.D. of innovations           4935.175
Date:                Tue, 11 Apr 2017   AIC                         152492.758
Time:                        21:19:27   BIC                         152534.439
Sample:                    02-05-2010   HQIC                        152507.054
                         - 10-08-2010                                         
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
const               1.372e+04   2802.080      4.896      0.000      8226.272  1.92e+04
ar.L1.Weekly_Sales     1.5133      0.067     22.755      0.000         1.383     1.644
ar.L2.Weekly_Sales  

#### Finally, compute an ARIMA model to improve your prediction error - iterate on the p, q, and parameters comparing the model's performance.

In [ ]:
# TODO